# Sparkify Project Workspace
This workspace contains a tiny subset (128MB) of the full dataset available (12GB). Feel free to use this workspace to build your project, or to explore a smaller subset with Spark before deploying your cluster on the cloud. Instructions for setting up your Spark cluster is included in the last lesson of the Extracurricular Spark Course content.

You can follow the steps below to guide your data analysis and model building portion of this project.

In [8]:
# import libraries
from pyspark.sql import SparkSession
from pyspark.ml.feature import RegexTokenizer, CountVectorizer, \
    IDF, StringIndexer, RegexTokenizer, VectorAssembler, Normalizer, StandardScaler
import pyspark.sql.functions as F
from pyspark.sql.types import IntegerType, StringType, FloatType
from pyspark.ml.feature import RegexTokenizer, VectorAssembler, Normalizer, StandardScaler
from pyspark.sql.functions import udf
from pyspark.sql.functions import desc, asc
from pyspark.sql.functions import sum as Fsum
from pyspark.sql.functions import concat, lit, avg, split, isnan, isnull, when, count, col, \
sum, mean, stddev, min, max, countDistinct,approx_count_distinct, size, collect_set, round
from pyspark.sql import Window


import seaborn as sns
import re
from datetime import datetime
import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt

from pyspark.ml.feature import StringIndexer, VectorAssembler, Normalizer, StandardScaler, MinMaxScaler
from pyspark.ml.feature import OneHotEncoder, StringIndexer, OneHotEncoderEstimator
from pyspark.ml.classification import LogisticRegression, GBTClassifier, NaiveBayes, RandomForestClassifier

from pyspark.ml import Pipeline
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.mllib.evaluation import BinaryClassificationMetrics, MulticlassMetrics

In [9]:
# create a Spark session
spark = SparkSession\
    .builder\
    .appName("SmallSparkify")\
    .getOrCreate()

# Load and Clean Dataset
In this workspace, the mini-dataset file is `mini_sparkify_event_data.json`. Load and clean the dataset, checking for invalid or missing data - for example, records without userids or sessionids. 

In [3]:
df = spark.read.options(header=True).json("mini_sparkify_event_data.json")

In [4]:
df.printSchema()
df.persist()

root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: long (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)



DataFrame[artist: string, auth: string, firstName: string, gender: string, itemInSession: bigint, lastName: string, length: double, level: string, location: string, method: string, page: string, registration: bigint, sessionId: bigint, song: string, status: bigint, ts: bigint, userAgent: string, userId: string]

In [5]:
df.take(1)

[Row(artist='Martha Tilston', auth='Logged In', firstName='Colin', gender='M', itemInSession=50, lastName='Freeman', length=277.89016, level='paid', location='Bakersfield, CA', method='PUT', page='NextSong', registration=1538173362000, sessionId=29, song='Rockpools', status=200, ts=1538352117000, userAgent='Mozilla/5.0 (Windows NT 6.1; WOW64; rv:31.0) Gecko/20100101 Firefox/31.0', userId='30')]

# Exploratory Data Analysis
When you're working with the full dataset, perform EDA by loading a small subset of the data and doing basic manipulations within Spark. In this workspace, you are already provided a small subset of data you can explore.

### Define Churn

Once you've done some preliminary analysis, create a column `Churn` to use as the label for your model. I suggest using the `Cancellation Confirmation` events to define your churn, which happen for both paid and free users. As a bonus task, you can also look into the `Downgrade` events.

### Explore Data
Once you've defined churn, perform some exploratory data analysis to observe the behavior for users who stayed vs users who churned. You can start by exploring aggregates on these two groups of users, observing how much of a specific action they experienced per a certain time unit or number of songs played.

### Basic Data Cleaning
Drop rows having whitespace in userId column

In [6]:
for column in df.columns:
    # check for both nan and null values 
    data_nan = df.where(isnan(df[column])).count()
    data_null = df.where(isnull(df[column])).count()
    data_whitespace = df.where(df[column]=='').count()
    print(column, "has {},{},{} nan, null or whitespace".format(data_nan, data_null, data_whitespace))
    
print(df.count())

artist has 0,58392,0 nan, null or whitespace
auth has 0,0,0 nan, null or whitespace
firstName has 0,8346,0 nan, null or whitespace
gender has 0,8346,0 nan, null or whitespace
itemInSession has 0,0,0 nan, null or whitespace
lastName has 0,8346,0 nan, null or whitespace
length has 0,58392,0 nan, null or whitespace
level has 0,0,0 nan, null or whitespace
location has 0,8346,0 nan, null or whitespace
method has 0,0,0 nan, null or whitespace
page has 0,0,0 nan, null or whitespace
registration has 0,8346,0 nan, null or whitespace
sessionId has 0,0,0 nan, null or whitespace
song has 0,58392,0 nan, null or whitespace
status has 0,0,0 nan, null or whitespace
ts has 0,0,0 nan, null or whitespace
userAgent has 0,8346,0 nan, null or whitespace
userId has 0,0,8346 nan, null or whitespace
286500


In [7]:
df = df.filter(df["userId"] != "")
print(df.count())

for column in df.columns:
    # check for both nan and null values 
    data_nan = df.where(isnan(df[column])).count()
    data_null = df.where(isnull(df[column])).count()
    data_whitespace = df.where(df[column]=='').count()
    print(column, "has {},{},{} nan, null or whitespace".format(data_nan, data_null, data_whitespace))

278154
artist has 0,50046,0 nan, null or whitespace
auth has 0,0,0 nan, null or whitespace
firstName has 0,0,0 nan, null or whitespace
gender has 0,0,0 nan, null or whitespace
itemInSession has 0,0,0 nan, null or whitespace
lastName has 0,0,0 nan, null or whitespace
length has 0,50046,0 nan, null or whitespace
level has 0,0,0 nan, null or whitespace
location has 0,0,0 nan, null or whitespace
method has 0,0,0 nan, null or whitespace
page has 0,0,0 nan, null or whitespace
registration has 0,0,0 nan, null or whitespace
sessionId has 0,0,0 nan, null or whitespace
song has 0,50046,0 nan, null or whitespace
status has 0,0,0 nan, null or whitespace
ts has 0,0,0 nan, null or whitespace
userAgent has 0,0,0 nan, null or whitespace
userId has 0,0,0 nan, null or whitespace


### Create churn column

In [8]:
# Make a list of ids who churned
churn_id_list = df[df.page == "Cancellation Confirmation"].select("userId").distinct().collect()
churn_id_list = [x['userId'] for x in churn_id_list]
# print(churn_id_list)

create_churn_udf = udf(lambda userid: 1 if userid in churn_id_list else 0, IntegerType())
df = df.withColumn("churn", create_churn_udf(df.userId))
df.where(df.churn == 1).distinct().take(1)

[Row(artist='Dwight Yoakam', auth='Logged In', firstName='Alexi', gender='F', itemInSession=131, lastName='Warren', length=239.3073, level='paid', location='Spokane-Spokane Valley, WA', method='PUT', page='NextSong', registration=1532482662000, sessionId=260, song="You're The One", status=200, ts=1538421792000, userAgent='Mozilla/5.0 (Windows NT 6.1; WOW64; rv:32.0) Gecko/20100101 Firefox/32.0', userId='54', churn=1)]

### Create new userAgent column

In [9]:
# Create new userAgent column
def create_new_agent(userAgent):
    if userAgent == None:
        computer_os = None
    
    else:
        computer_os = userAgent.split()[1]
        computer_os = re.sub(r'[\W\s]','' ,computer_os)
    
    return  computer_os

create_new_agent_udf = udf(create_new_agent, StringType())
df = df.withColumn("os_system", create_new_agent_udf(df.userAgent))
df.where(df.os_system == 'iPhone').distinct().take(1)

[Row(artist=None, auth='Logged In', firstName='Parker', gender='M', itemInSession=74, lastName='Williams', length=None, level='free', location='San Francisco-Oakland-Hayward, CA', method='PUT', page='Add to Playlist', registration=1537188538000, sessionId=60, song=None, status=200, ts=1538464482000, userAgent='"Mozilla/5.0 (iPhone; CPU iPhone OS 7_1_1 like Mac OS X) AppleWebKit/537.51.2 (KHTML, like Gecko) Version/7.0 Mobile/11D201 Safari/9537.53"', userId='61', churn=0, os_system='iPhone')]

### Create new location column

In [10]:
# Create new location column
def create_new_location(location):
    if location == None:
        location_first = None

    else:
        location_first = location.split(',')[-1].split('-')[0].strip()
    
    return location_first

create_new_location_udf = udf(create_new_location, StringType())
df = df.withColumn("location_first", create_new_location_udf(df.location))
df.where(df.location_first == 'MA').distinct().take(1)

[Row(artist='Randy Travis', auth='Logged In', firstName='Micah', gender='M', itemInSession=22, lastName='Long', length=278.85669, level='free', location='Boston-Cambridge-Newton, MA-NH', method='PUT', page='NextSong', registration=1538331630000, sessionId=346, song="'Til I'm Dead And Gone (Album Version)", status=200, ts=1538522965000, userAgent='"Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/37.0.2062.103 Safari/537.36"', userId='9', churn=0, os_system='Windows', location_first='MA')]

### Create total number of sessionId

In [11]:
print("Max session id of userId 9: ",df.where(df.userId == '9').groupby().max('sessionId').collect()[0]["max(sessionId)"])
print(len(df.select(['userId', "sessionId"]).where(df.userId == '9').distinct().collect()))
# Create total number of sessionId
df.groupBy("userId").agg(countDistinct("sessionId").alias("Num_sessionId")).collect()

Max session id of userId 9:  2356
31


[Row(userId='100010', Num_sessionId=7),
 Row(userId='200002', Num_sessionId=6),
 Row(userId='125', Num_sessionId=1),
 Row(userId='51', Num_sessionId=10),
 Row(userId='124', Num_sessionId=29),
 Row(userId='7', Num_sessionId=7),
 Row(userId='54', Num_sessionId=37),
 Row(userId='15', Num_sessionId=15),
 Row(userId='155', Num_sessionId=6),
 Row(userId='100014', Num_sessionId=6),
 Row(userId='132', Num_sessionId=16),
 Row(userId='154', Num_sessionId=3),
 Row(userId='101', Num_sessionId=10),
 Row(userId='11', Num_sessionId=16),
 Row(userId='300017', Num_sessionId=63),
 Row(userId='138', Num_sessionId=15),
 Row(userId='29', Num_sessionId=34),
 Row(userId='69', Num_sessionId=9),
 Row(userId='100021', Num_sessionId=5),
 Row(userId='42', Num_sessionId=47),
 Row(userId='112', Num_sessionId=10),
 Row(userId='87', Num_sessionId=25),
 Row(userId='73', Num_sessionId=6),
 Row(userId='200010', Num_sessionId=6),
 Row(userId='64', Num_sessionId=3),
 Row(userId='3', Num_sessionId=4),
 Row(userId='113', Nu

In [12]:
# Create total number of sessionId column
w = Window.partitionBy(df.userId)
df = df.withColumn('total_sessionId', size(collect_set('sessionId').over(w)))
df[df.userId == '140'].take(1)

[Row(artist=None, auth='Logged In', firstName='Lucero', gender='F', itemInSession=0, lastName='Reed', length=None, level='free', location='Louisville/Jefferson County, KY-IN', method='GET', page='Home', registration=1536642109000, sessionId=139, song=None, status=200, ts=1538406127000, userAgent='"Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.125 Safari/537.36"', userId='140', churn=0, os_system='Windows', location_first='KY', total_sessionId=71)]

### total itemInSession

In [13]:
print("total itemInSession of userId 9: ", len(df.select(['userId','sessionId', 'itemInSession']).where(df.userId == '9').collect()))
df.select(['userId','sessionId', 'itemInSession']).where(df.userId == '9').collect()

total itemInSession of userId 9:  3191


[Row(userId='9', sessionId=8, itemInSession=79),
 Row(userId='9', sessionId=8, itemInSession=80),
 Row(userId='9', sessionId=8, itemInSession=81),
 Row(userId='9', sessionId=8, itemInSession=82),
 Row(userId='9', sessionId=8, itemInSession=83),
 Row(userId='9', sessionId=8, itemInSession=84),
 Row(userId='9', sessionId=8, itemInSession=85),
 Row(userId='9', sessionId=8, itemInSession=86),
 Row(userId='9', sessionId=8, itemInSession=87),
 Row(userId='9', sessionId=8, itemInSession=88),
 Row(userId='9', sessionId=8, itemInSession=89),
 Row(userId='9', sessionId=8, itemInSession=90),
 Row(userId='9', sessionId=8, itemInSession=91),
 Row(userId='9', sessionId=8, itemInSession=92),
 Row(userId='9', sessionId=8, itemInSession=93),
 Row(userId='9', sessionId=8, itemInSession=94),
 Row(userId='9', sessionId=8, itemInSession=95),
 Row(userId='9', sessionId=8, itemInSession=96),
 Row(userId='9', sessionId=8, itemInSession=97),
 Row(userId='9', sessionId=8, itemInSession=98),
 Row(userId='9', ses

In [14]:
# Create total number of itemInSession column
w = Window.partitionBy(df.userId)
df = df.withColumn('total_itemInSession', count('itemInSession').over(w))
df[df.userId == '9'].take(1)

[Row(artist='Five Iron Frenzy', auth='Logged In', firstName='Micah', gender='M', itemInSession=79, lastName='Long', length=236.09424, level='free', location='Boston-Cambridge-Newton, MA-NH', method='PUT', page='NextSong', registration=1538331630000, sessionId=8, song='Canada', status=200, ts=1538352180000, userAgent='"Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/37.0.2062.103 Safari/537.36"', userId='9', churn=0, os_system='Windows', location_first='MA', total_sessionId=31, total_itemInSession=3191)]

### Number of thumb up/ thumb down/ advert/ add friend/ upgrade/ downgrade/ error

In [15]:
# Create dataframes to be joined later (thumup, thumbdown, advert, addfriend, upgrade, downgrade, error)
w = Window.partitionBy('userId')
thumbsup_df = df.where(df.page == 'Thumbs Up').groupBy("userId").agg(count("page").alias("total_thumbsup"))
thumbsdown_df = df.where(df.page == 'Thumbs Down').groupBy("userId").agg(count("page").alias("total_thumbsdown"))
advert_df =  df.where(df.page == 'Roll Advert').groupBy("userId").agg(count("page").alias("total_rolladvert"))
addfriend_df =  df.where(df.page == 'Add Friend').groupBy("userId").agg(count("page").alias("total_addfriend"))
addplaylist_df =  df.where(df.page == 'Add to Playlist').groupBy("userId").agg(count("page").alias("total_addplaylist"))
sub_upgrade_df = df.where(df.page == 'Submit Upgrade').groupBy("userId").agg(count("page").alias("total_sub_upgrade"))
sub_downgrade_df = df.where(df.page == 'Submit Downgrade').groupBy("userId").agg(count("page").alias("total_sub_downgrade"))
error_df = df.where(df.page == 'Error').groupBy("userId").agg(count("page").alias("total_error"))
logout_df = df.where(df.page == 'Logout').groupBy("userId").agg(count("page").alias("total_logout"))
thumbsup_df.collect()

[Row(userId='100010', total_thumbsup=17),
 Row(userId='200002', total_thumbsup=21),
 Row(userId='51', total_thumbsup=100),
 Row(userId='124', total_thumbsup=171),
 Row(userId='7', total_thumbsup=7),
 Row(userId='54', total_thumbsup=163),
 Row(userId='15', total_thumbsup=81),
 Row(userId='155', total_thumbsup=58),
 Row(userId='132', total_thumbsup=96),
 Row(userId='154', total_thumbsup=11),
 Row(userId='100014', total_thumbsup=17),
 Row(userId='101', total_thumbsup=86),
 Row(userId='11', total_thumbsup=40),
 Row(userId='138', total_thumbsup=95),
 Row(userId='300017', total_thumbsup=303),
 Row(userId='29', total_thumbsup=154),
 Row(userId='69', total_thumbsup=72),
 Row(userId='100021', total_thumbsup=11),
 Row(userId='42', total_thumbsup=166),
 Row(userId='112', total_thumbsup=9),
 Row(userId='87', total_thumbsup=32),
 Row(userId='73', total_thumbsup=14),
 Row(userId='64', total_thumbsup=4),
 Row(userId='200010', total_thumbsup=14),
 Row(userId='3', total_thumbsup=14),
 Row(userId='30', 

### Create last_access_time , first_access_time columns 

In [16]:
# Create time after id creation (in hour) columns
w = Window.partitionBy('userId')
df = df.withColumn('last_access_time', max('ts').over(w))
df = df.withColumn('first_access_time', min('ts').over(w))

df.take(1)

[Row(artist='Sleeping With Sirens', auth='Logged In', firstName='Darianna', gender='F', itemInSession=0, lastName='Carpenter', length=202.97098, level='free', location='Bridgeport-Stamford-Norwalk, CT', method='PUT', page='NextSong', registration=1538016340000, sessionId=31, song='Captain Tyin Knots VS Mr Walkway (No Way)', status=200, ts=1539003534000, userAgent='"Mozilla/5.0 (iPhone; CPU iPhone OS 7_1_2 like Mac OS X) AppleWebKit/537.51.2 (KHTML, like Gecko) Version/7.0 Mobile/11D257 Safari/9537.53"', userId='100010', churn=0, os_system='iPhone', location_first='CT', total_sessionId=7, total_itemInSession=381, last_access_time=1542823952000, first_access_time=1539003534000)]

### Create last_level column

In [17]:
# Create last_level column
# Create dataframe to be joined later (last_level)
df = df.withColumn('last_level',when(df.last_access_time == df.ts, df.level))
df.dtypes
last_level_df = df[df.last_level != 'None'].select("userId","last_level")
last_level_df.take(1)


[Row(userId='100010', last_level='free')]

In [18]:
df[df.userId == '10'].take(1)

[Row(artist='Sea Wolf', auth='Logged In', firstName='Brycen', gender='M', itemInSession=0, lastName='Bradley', length=265.53424, level='paid', location='Laurel, MS', method='PUT', page='NextSong', registration=1538159495000, sessionId=9, song='Ses Monuments', status=200, ts=1538965220000, userAgent='"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.143 Safari/537.36"', userId='10', churn=0, os_system='Macintosh', location_first='MS', total_sessionId=6, total_itemInSession=795, last_access_time=1542631788000, first_access_time=1538965220000, last_level=None)]

## Create time after id creation

In [19]:
# Create time difference column
def calculate_time_after_id_creation(last_time, first_time):
    last_access_datetime = datetime.utcfromtimestamp(last_time / 1000)
    first_access_datetime = datetime.utcfromtimestamp(first_time / 1000)
    time_after_id_creation = last_access_datetime - first_access_datetime
    result = time_after_id_creation.total_seconds()/3600
    
    return result

calculate_time_after_id_creation_udf = udf(calculate_time_after_id_creation, FloatType())
df = df.withColumn("time_after_id_creation(hour)", calculate_time_after_id_creation_udf(df.last_access_time, df.first_access_time))
df = df.withColumn("time_after_id_creation(hour)", round(col('time_after_id_creation(hour)')/1, 2))
df[df.userId == '9'].take(1)

[Row(artist='Five Iron Frenzy', auth='Logged In', firstName='Micah', gender='M', itemInSession=79, lastName='Long', length=236.09424, level='free', location='Boston-Cambridge-Newton, MA-NH', method='PUT', page='NextSong', registration=1538331630000, sessionId=8, song='Canada', status=200, ts=1538352180000, userAgent='"Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/37.0.2062.103 Safari/537.36"', userId='9', churn=0, os_system='Windows', location_first='MA', total_sessionId=31, total_itemInSession=3191, last_access_time=1543587552000, first_access_time=1538352180000, last_level=None, time_after_id_creation(hour)=1454.27)]

### Create total spent time

In [20]:
# Create total spent time
# Create dataframe to be joined later (spent_time_df)
spent_time_df = df.groupBy("userId", "sessionId").agg(max('ts').alias("max_ts_session"), min('ts').alias("min_ts_session")).orderBy("sessionId", ascending=True)

spent_time_df[spent_time_df.userId == '9'].collect()

def calculate_time_inSession(max_ts_session, min_ts_session):
    max_ts_session_datetime = datetime.utcfromtimestamp(max_ts_session / 1000)
    min_ts_session_datetime = datetime.utcfromtimestamp(min_ts_session / 1000)
    spent_time_session = max_ts_session_datetime - min_ts_session_datetime
    result = spent_time_session.total_seconds()/3600

    return result

calculate_time_inSession_udf = udf(calculate_time_inSession, FloatType())
spent_time_df = spent_time_df.withColumn("time_spent_Session_hour", 
                                             calculate_time_inSession_udf(spent_time_df.max_ts_session, spent_time_df.min_ts_session))

w = Window.partitionBy(spent_time_df.userId)
spent_time_df = spent_time_df.withColumn("time_spent_Total_hour", sum(spent_time_df.time_spent_Session_hour).over(w))
spent_time_df[spent_time_df.userId == '9'].head(1)

[Row(userId='9', sessionId=8, max_ts_session=1538360228000, min_ts_session=1538352180000, time_spent_Session_hour=2.235555648803711, time_spent_Total_hour=182.16472381353378)]

In [21]:
spent_time_df = spent_time_df.drop('max_ts_session','min_ts_session')
spent_time_df = spent_time_df.withColumn('time_spent_Session_hour', round(col('time_spent_Session_hour')/1, 2))
spent_time_df = spent_time_df.withColumn('time_spent_Total_hour', round(col('time_spent_Total_hour')/1, 2))
spent_time_df_only_total = spent_time_df.drop('time_spent_Session_hour','sessionId').distinct()
spent_time_df_only_total.take(2)

[Row(userId='100010', time_spent_Total_hour=18.02),
 Row(userId='200002', time_spent_Total_hour=26.64)]

In [22]:
spent_time_df_only_total.count()

225

### Create a column about the number of listening of top100 artist in all time (steady seller)

In [23]:
# convert timestamp to date (string)
def create_numWeek(ts):
    return datetime.utcfromtimestamp(ts / 1000).strftime("%V")
def create_numMonth(ts):
    return datetime.utcfromtimestamp(ts / 1000).strftime("%m")
def create_numYear(ts):
    return datetime.utcfromtimestamp(ts / 1000).strftime("%Y")

create_numWeek_udf = udf(create_numWeek, StringType())
create_numMonth_udf = udf(create_numMonth, StringType())
create_numYear_udf = udf(create_numYear, StringType())
df = df.withColumn('num_week', create_numWeek_udf(col('ts')))
df = df.withColumn('num_month', create_numMonth_udf(col('ts')))
df = df.withColumn('num_year', create_numYear_udf(col('ts')))
df.take(1)

[Row(artist='Sleeping With Sirens', auth='Logged In', firstName='Darianna', gender='F', itemInSession=0, lastName='Carpenter', length=202.97098, level='free', location='Bridgeport-Stamford-Norwalk, CT', method='PUT', page='NextSong', registration=1538016340000, sessionId=31, song='Captain Tyin Knots VS Mr Walkway (No Way)', status=200, ts=1539003534000, userAgent='"Mozilla/5.0 (iPhone; CPU iPhone OS 7_1_2 like Mac OS X) AppleWebKit/537.51.2 (KHTML, like Gecko) Version/7.0 Mobile/11D257 Safari/9537.53"', userId='100010', churn=0, os_system='iPhone', location_first='CT', total_sessionId=7, total_itemInSession=381, last_access_time=1542823952000, first_access_time=1539003534000, last_level=None, time_after_id_creation(hour)=1061.23, num_week='41', num_month='10', num_year='2018')]

In [24]:
# Make a top_100 alltime artist list
tmp_list = df.where(df.artist != "").groupby("artist").count().sort(col("count").desc()).collect()[0:100]
top_100_alltime_artist_list = [row["artist"] for row in tmp_list]
top_100_alltime_artist_list

# Make udf to set 1 at churn column in every row of the chrun users 
def create_top100_alltime(artist):
    if artist in top_100_alltime_artist_list:
        return 1
    else:
        return 0

create_top100_alltime_udf = udf(create_top100_alltime, IntegerType())
df = df.withColumn("top100_artist_alltime", create_top100_alltime_udf(df.artist))
# df.take(2)

In [25]:
# Count total number of top 100
w = Window.partitionBy(df.userId)
df = df.withColumn('total_Top100_artist_alltime', sum('top100_artist_alltime').over(w))
df.take(2)

[Row(artist='Sleeping With Sirens', auth='Logged In', firstName='Darianna', gender='F', itemInSession=0, lastName='Carpenter', length=202.97098, level='free', location='Bridgeport-Stamford-Norwalk, CT', method='PUT', page='NextSong', registration=1538016340000, sessionId=31, song='Captain Tyin Knots VS Mr Walkway (No Way)', status=200, ts=1539003534000, userAgent='"Mozilla/5.0 (iPhone; CPU iPhone OS 7_1_2 like Mac OS X) AppleWebKit/537.51.2 (KHTML, like Gecko) Version/7.0 Mobile/11D257 Safari/9537.53"', userId='100010', churn=0, os_system='iPhone', location_first='CT', total_sessionId=7, total_itemInSession=381, last_access_time=1542823952000, first_access_time=1539003534000, last_level=None, time_after_id_creation(hour)=1061.23, num_week='41', num_month='10', num_year='2018', top100_artist_alltime=0, total_Top100_artist_alltime=61),
 Row(artist='Francesca Battistelli', auth='Logged In', firstName='Darianna', gender='F', itemInSession=1, lastName='Carpenter', length=196.54485, level='f

### Create a column about the number of listening of top100 songs in a week (best seller)

In [26]:
# Make a dictionary of a best seller song list of each week
tmp_list = df.select("num_week").distinct().sort("num_week").collect()
available_week_list = [row["num_week"] for row in tmp_list]
available_week_list

['40', '41', '42', '43', '44', '45', '46', '47', '48', '49']

In [27]:
# Make a dictionary of a best seller song list of each week
start_time = datetime.now()
def create_dict_top100_song_week(available_week_list):
    dict_top100_song_week = dict()
    for week in available_week_list:
        top_100_week_song_list = df.where((df.artist != "") & (df.num_week == week)).groupby("song","num_week").count()\
        .sort(col("num_week"), col("count").desc()).collect()[0:100]
        top_100_week_song_list = [row['song'] for row in top_100_week_song_list]
        dict_top100_song_week[week] = top_100_week_song_list
        
    return dict_top100_song_week
        
dict_top100_song_week = create_dict_top100_song_week(available_week_list)
print(datetime.now() - start_time)

0:01:05.921084


In [28]:
# Make a top_100_song_week list
start_time = datetime.now()
def create_top100_song_week(song, num_week):
    if song in dict_top100_song_week[num_week]:
        return 1
    else:
        return 0

create_top100_song_week_udf = udf(create_top100_song_week, IntegerType())
df = df.withColumn("top100_song_week", create_top100_song_week_udf(df.song, df.num_week))
print(datetime.now() - start_time)
df.take(2)

0:00:00.071077


[Row(artist='Sleeping With Sirens', auth='Logged In', firstName='Darianna', gender='F', itemInSession=0, lastName='Carpenter', length=202.97098, level='free', location='Bridgeport-Stamford-Norwalk, CT', method='PUT', page='NextSong', registration=1538016340000, sessionId=31, song='Captain Tyin Knots VS Mr Walkway (No Way)', status=200, ts=1539003534000, userAgent='"Mozilla/5.0 (iPhone; CPU iPhone OS 7_1_2 like Mac OS X) AppleWebKit/537.51.2 (KHTML, like Gecko) Version/7.0 Mobile/11D257 Safari/9537.53"', userId='100010', churn=0, os_system='iPhone', location_first='CT', total_sessionId=7, total_itemInSession=381, last_access_time=1542823952000, first_access_time=1539003534000, last_level=None, time_after_id_creation(hour)=1061.23, num_week='41', num_month='10', num_year='2018', top100_artist_alltime=0, total_Top100_artist_alltime=61, top100_song_week=0),
 Row(artist='Francesca Battistelli', auth='Logged In', firstName='Darianna', gender='F', itemInSession=1, lastName='Carpenter', length

In [29]:
# Count total number of top 100_song_week
w = Window.partitionBy(df.userId)
df = df.withColumn('total_Top100_song_week', sum('top100_song_week').over(w))

In [30]:
df.head(3)

[Row(artist='Sleeping With Sirens', auth='Logged In', firstName='Darianna', gender='F', itemInSession=0, lastName='Carpenter', length=202.97098, level='free', location='Bridgeport-Stamford-Norwalk, CT', method='PUT', page='NextSong', registration=1538016340000, sessionId=31, song='Captain Tyin Knots VS Mr Walkway (No Way)', status=200, ts=1539003534000, userAgent='"Mozilla/5.0 (iPhone; CPU iPhone OS 7_1_2 like Mac OS X) AppleWebKit/537.51.2 (KHTML, like Gecko) Version/7.0 Mobile/11D257 Safari/9537.53"', userId='100010', churn=0, os_system='iPhone', location_first='CT', total_sessionId=7, total_itemInSession=381, last_access_time=1542823952000, first_access_time=1539003534000, last_level=None, time_after_id_creation(hour)=1061.23, num_week='41', num_month='10', num_year='2018', top100_artist_alltime=0, total_Top100_artist_alltime=61, top100_song_week=0, total_Top100_song_week=31),
 Row(artist='Francesca Battistelli', auth='Logged In', firstName='Darianna', gender='F', itemInSession=1, l

### Create new df to join other dataframes created above

In [31]:
df.printSchema()

root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: long (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)
 |-- churn: integer (nullable = true)
 |-- os_system: string (nullable = true)
 |-- location_first: string (nullable = true)
 |-- total_sessionId: integer (nullable = false)
 |-- total_itemInSession: long (nullable = false)
 |-- last_access_time: long (nullable = true)
 |-- first_access_time: long (nullab

In [32]:
df_new = df.select("userId",'gender', 'churn', 'os_system', 'location_first', 'total_sessionId', 'total_itemInSession',"time_after_id_creation(hour)", 
                   "total_Top100_artist_alltime", "total_Top100_song_week").dropna().drop_duplicates()
# print(df_new.count())
# print(df.count())
# df_new.head(3)

In [33]:
df_new = df_new.join(thumbsup_df, 'userId', how='left').distinct()
df_new = df_new.join(thumbsdown_df, 'userId', how='left').distinct()
df_new = df_new.join(advert_df, 'userId', how='left').distinct()
df_new = df_new.join(addfriend_df, 'userId', how='left').distinct()
df_new = df_new.join(addplaylist_df, 'userId', how='left').distinct()
df_new = df_new.join(sub_upgrade_df, 'userId', how='left').distinct()
df_new = df_new.join(sub_downgrade_df, 'userId', how='left').distinct()
df_new = df_new.join(error_df, 'userId', how='left').distinct()
df_new = df_new.join(logout_df, 'userId', how='left').distinct()
df_new = df_new.fillna(0, subset=['total_thumbsup','total_thumbsdown', 'total_rolladvert', 'total_addfriend', 'total_addplaylist', 'total_sub_upgrade', 
                                 'total_sub_downgrade', 'total_error', 'total_logout'])
df_new.head(3)

[Row(userId='100010', gender='F', churn=0, os_system='iPhone', location_first='CT', total_sessionId=7, total_itemInSession=381, time_after_id_creation(hour)=1061.23, total_Top100_artist_alltime=61, total_Top100_song_week=31, total_thumbsup=17, total_thumbsdown=5, total_rolladvert=52, total_addfriend=4, total_addplaylist=7, total_sub_upgrade=0, total_sub_downgrade=0, total_error=0, total_logout=5),
 Row(userId='200002', gender='M', churn=0, os_system='iPhone', location_first='IL', total_sessionId=6, total_itemInSession=474, time_after_id_creation(hour)=1091.92, total_Top100_artist_alltime=82, total_Top100_song_week=32, total_thumbsup=21, total_thumbsdown=6, total_rolladvert=7, total_addfriend=4, total_addplaylist=8, total_sub_upgrade=1, total_sub_downgrade=0, total_error=0, total_logout=5),
 Row(userId='125', gender='M', churn=1, os_system='Macintosh', location_first='TX', total_sessionId=1, total_itemInSession=11, time_after_id_creation(hour)=0.49, total_Top100_artist_alltime=2, total_

In [34]:
df_new = df_new.join(spent_time_df_only_total, 'userId', how='left').distinct()
df_new = df_new.join(last_level_df, 'userId', how='left').distinct()
# print(df_new.count())
df_new.head(4)

[Row(userId='100010', gender='F', churn=0, os_system='iPhone', location_first='CT', total_sessionId=7, total_itemInSession=381, time_after_id_creation(hour)=1061.23, total_Top100_artist_alltime=61, total_Top100_song_week=31, total_thumbsup=17, total_thumbsdown=5, total_rolladvert=52, total_addfriend=4, total_addplaylist=7, total_sub_upgrade=0, total_sub_downgrade=0, total_error=0, total_logout=5, time_spent_Total_hour=18.02, last_level='free'),
 Row(userId='200002', gender='M', churn=0, os_system='iPhone', location_first='IL', total_sessionId=6, total_itemInSession=474, time_after_id_creation(hour)=1091.92, total_Top100_artist_alltime=82, total_Top100_song_week=32, total_thumbsup=21, total_thumbsdown=6, total_rolladvert=7, total_addfriend=4, total_addplaylist=8, total_sub_upgrade=1, total_sub_downgrade=0, total_error=0, total_logout=5, time_spent_Total_hour=26.64, last_level='paid'),
 Row(userId='125', gender='M', churn=1, os_system='Macintosh', location_first='TX', total_sessionId=1, 

In [35]:
df_new.cache()
df_new.head(2)

[Row(userId='100010', gender='F', churn=0, os_system='iPhone', location_first='CT', total_sessionId=7, total_itemInSession=381, time_after_id_creation(hour)=1061.23, total_Top100_artist_alltime=61, total_Top100_song_week=31, total_thumbsup=17, total_thumbsdown=5, total_rolladvert=52, total_addfriend=4, total_addplaylist=7, total_sub_upgrade=0, total_sub_downgrade=0, total_error=0, total_logout=5, time_spent_Total_hour=18.02, last_level='free'),
 Row(userId='200002', gender='M', churn=0, os_system='iPhone', location_first='IL', total_sessionId=6, total_itemInSession=474, time_after_id_creation(hour)=1091.92, total_Top100_artist_alltime=82, total_Top100_song_week=32, total_thumbsup=21, total_thumbsdown=6, total_rolladvert=7, total_addfriend=4, total_addplaylist=8, total_sub_upgrade=1, total_sub_downgrade=0, total_error=0, total_logout=5, time_spent_Total_hour=26.64, last_level='paid')]

In [36]:
# df_new.take(1)
df_new1 = df_new.withColumn("time_spent_Total_day", round(col('time_spent_Total_hour')/24, 2))
df_new1 = df_new1.withColumn("time_after_id_creation(day)", round(col('time_after_id_creation(hour)')/24, 2))
df_new1 = df_new1.withColumn("avg_total_sessionId_afterCreation", round(col('total_sessionId') / col('time_after_id_creation(day)'), 2))
df_new1 = df_new1.withColumn("avg_itemInSession_afterCreation", round(col('total_itemInSession')/col('time_after_id_creation(day)'), 2))
df_new1 = df_new1.withColumn("avg_thumbsup_afterCreation", round(col('total_thumbsup')/col('time_after_id_creation(day)'), 2))
df_new1 = df_new1.withColumn("avg_thumbsdown_afterCreation", round((col('total_thumbsdown')/col('time_after_id_creation(day)')), 2))
df_new1 = df_new1.withColumn("avg_rolladvert_afterCreation", round((col('total_rolladvert')/col('time_after_id_creation(day)')), 2))
df_new1 = df_new1.withColumn("avg_addfriend_afterCreation", round((col('total_addfriend')/col('time_after_id_creation(day)')), 2))
df_new1 = df_new1.withColumn("avg_addplaylist_afterCreation", round((col('total_addplaylist')/col('time_after_id_creation(day)')), 2))
df_new1 = df_new1.withColumn("avg_error_afterCreation", round((col('total_error')/col('time_after_id_creation(day)')), 2))
df_new1 = df_new1.withColumn("avg_logout_afterCreation", round((col('total_logout')/col('time_after_id_creation(day)')), 2))
df_new1 = df_new1.withColumn("avg_total_Top100_artist_alltime", round((col('total_Top100_artist_alltime')/col('time_after_id_creation(day)')), 2))
df_new1 = df_new1.withColumn("avg_total_Top100_song_week", round((col('total_Top100_song_week')/col('time_after_id_creation(day)')), 2))
df_new1.cache()

df_new1.head(3)

[Row(userId='100010', gender='F', churn=0, os_system='iPhone', location_first='CT', total_sessionId=7, total_itemInSession=381, time_after_id_creation(hour)=1061.23, total_Top100_artist_alltime=61, total_Top100_song_week=31, total_thumbsup=17, total_thumbsdown=5, total_rolladvert=52, total_addfriend=4, total_addplaylist=7, total_sub_upgrade=0, total_sub_downgrade=0, total_error=0, total_logout=5, time_spent_Total_hour=18.02, last_level='free', time_spent_Total_day=0.75, time_after_id_creation(day)=44.22, avg_total_sessionId_afterCreation=0.16, avg_itemInSession_afterCreation=8.62, avg_thumbsup_afterCreation=0.38, avg_thumbsdown_afterCreation=0.11, avg_rolladvert_afterCreation=1.18, avg_addfriend_afterCreation=0.09, avg_addplaylist_afterCreation=0.16, avg_error_afterCreation=0.0, avg_logout_afterCreation=0.11, avg_total_Top100_artist_alltime=1.38, avg_total_Top100_song_week=0.7),
 Row(userId='200002', gender='M', churn=0, os_system='iPhone', location_first='IL', total_sessionId=6, total

In [37]:
df_new1.head(1)

[Row(userId='100010', gender='F', churn=0, os_system='iPhone', location_first='CT', total_sessionId=7, total_itemInSession=381, time_after_id_creation(hour)=1061.23, total_Top100_artist_alltime=61, total_Top100_song_week=31, total_thumbsup=17, total_thumbsdown=5, total_rolladvert=52, total_addfriend=4, total_addplaylist=7, total_sub_upgrade=0, total_sub_downgrade=0, total_error=0, total_logout=5, time_spent_Total_hour=18.02, last_level='free', time_spent_Total_day=0.75, time_after_id_creation(day)=44.22, avg_total_sessionId_afterCreation=0.16, avg_itemInSession_afterCreation=8.62, avg_thumbsup_afterCreation=0.38, avg_thumbsdown_afterCreation=0.11, avg_rolladvert_afterCreation=1.18, avg_addfriend_afterCreation=0.09, avg_addplaylist_afterCreation=0.16, avg_error_afterCreation=0.0, avg_logout_afterCreation=0.11, avg_total_Top100_artist_alltime=1.38, avg_total_Top100_song_week=0.7)]

In [ ]:
print(df_new1.groupby('churn').agg(round(mean('avg_total_sessionId_afterCreation'),2)).collect())
print(df_new1.groupby('churn').agg(round(stddev('avg_total_sessionId_afterCreation'),2)).collect())
print("**"*20)
print(df_new1.groupby('churn').agg(round(mean('avg_itemInSession_afterCreation'),2)).collect())
print(df_new1.groupby('churn').agg(round(stddev('avg_itemInSession_afterCreation'),2)).collect())
print("**"*20)
print(df_new1.groupby('churn').agg(round(mean('avg_thumbsup_afterCreation'),2)).collect())
print(df_new1.groupby('churn').agg(round(stddev('avg_thumbsup_afterCreation'),2)).collect())
print("**"*20)
# print(df_new1.groupby('churn').agg(mean('avg_thumbsdown_perday')).collect())
# print("**"*20)
print(df_new1.groupby('churn').agg(round(mean('avg_rolladvert_afterCreation'),2)).collect())
print(df_new1.groupby('churn').agg(round(stddev('avg_rolladvert_afterCreation'),2)).collect())
print("**"*20)
# print(df_new1.groupby('churn').agg(mean('avg_addfriend_perday')).collect())
# print("**"*20)
# print(df_new1.groupby('churn').agg(mean('avg_addplaylist_perday')).collect())
# print("**"*20)
# print(df_new1.groupby('churn').agg(mean('avg_error_perday')).collect())
# print("**"*20)
# print(df_new1.groupby('churn').agg(mean('avg_logout_perday')).collect())
# print("**"*20)


[Row(churn=1, round(avg(avg_total_sessionId_afterCreation), 2)=2.89), Row(churn=0, round(avg(avg_total_sessionId_afterCreation), 2)=1.38)]
[Row(churn=1, round(stddev_samp(avg_total_sessionId_afterCreation), 2)=8.57), Row(churn=0, round(stddev_samp(avg_total_sessionId_afterCreation), 2)=8.53)]
****************************************
[Row(churn=1, round(avg(avg_itemInSession_afterCreation), 2)=107.05), Row(churn=0, round(avg(avg_itemInSession_afterCreation), 2)=47.73)]
[Row(churn=1, round(stddev_samp(avg_itemInSession_afterCreation), 2)=181.99), Row(churn=0, round(stddev_samp(avg_itemInSession_afterCreation), 2)=95.18)]
****************************************
[Row(churn=1, round(avg(avg_thumbsup_afterCreation), 2)=4.63), Row(churn=0, round(avg(avg_thumbsup_afterCreation), 2)=1.93)]
[Row(churn=1, round(stddev_samp(avg_thumbsup_afterCreation), 2)=12.75), Row(churn=0, round(stddev_samp(avg_thumbsup_afterCreation), 2)=4.0)]
****************************************


In [ ]:
# print(df_new1.groupBy(['churn','os_system']).count().sort("churn", ascending=False).collect())
for i in df_new1.groupBy(['churn','os_system']).count().sort("churn", ascending=False).collect():
    print(i)

In [ ]:
df_new1.groupBy(['churn','last_level','gender']).count().sort([ 'gender','last_level','churn'], ascending=False).collect()
pandas_df = df_new1.groupBy(['churn','last_level','gender']).count().sort([ 'gender','last_level','churn'], ascending=False).toPandas()
pandas_df.head(10)

In [ ]:
df_new.take(2)

# Feature Engineering
Once you've familiarized yourself with the data, build out the features you find promising to train your model on. To work with the full dataset, you can follow the following steps.
- Write a script to extract the necessary features from the smaller subset of data
- Ensure that your script is scalable, using the best practices discussed in Lesson 3
- Try your script on the full data set, debugging your script if necessary

If you are working in the classroom workspace, you can just extract features based on the small subset of data contained here. Be sure to transfer over this work to the larger dataset when you work on your Spark cluster.

## Create a function for feature extraction

In [13]:
def create_dataframe_forML(df):
    def subfunc_cleaning(df):
        # Remove no id rows
        df = df.filter(df["userId"] != "")
        
        return df
        
    def subfunc_featureCreation(df):
        # Make a list of ids who churned
        churn_id_list = df[df.page == "Cancellation Confirmation"].select("userId").distinct().collect()
        churn_id_list = [x['userId'] for x in churn_id_list]
        create_churn_udf = udf(lambda userid: 1 if userid in churn_id_list else 0, IntegerType())
        df = df.withColumn("churn", create_churn_udf(df.userId))

        # Create new userAgent column
        def create_new_agent(userAgent):
            if userAgent == None:
                computer_os = None
            else:
                computer_os = userAgent.split()[1]
                computer_os = re.sub(r'[\W\s]','' ,computer_os)

            return  computer_os
        create_new_agent_udf = udf(create_new_agent, StringType())
        df = df.withColumn("os_system", create_new_agent_udf(df.userAgent))

        # Create new location column
        def create_new_location(location):
            if location == None:
                location_first = None
            else:
                location_first = location.split(',')[-1].split('-')[0].strip()

            return location_first

        create_new_location_udf = udf(create_new_location, StringType())
        df = df.withColumn("location_first", create_new_location_udf(df.location))

        # Create total number of sessionId column
        w = Window.partitionBy(df.userId)
        df = df.withColumn('total_sessionId', size(collect_set('sessionId').over(w)))

        # Create total number of itemInSession column
        df = df.withColumn('total_itemInSession', count('itemInSession').over(w))


      

        # Create last_access_time,first_access_time columns
        df = df.withColumn('last_access_time', max('ts').over(w))
        df = df.withColumn('first_access_time', min('ts').over(w))

        # Create last_level column
        df = df.withColumn('last_level',when(df.last_access_time == df.ts, df.level))


        # Create time difference column
        def calculate_time_after_id_creation(last_time, first_time):
            last_access_datetime = datetime.utcfromtimestamp(last_time / 1000)
            first_access_datetime = datetime.utcfromtimestamp(first_time / 1000)
            time_after_id_creation = last_access_datetime - first_access_datetime
            result = time_after_id_creation.total_seconds()/3600

            return result

        calculate_time_after_id_creation_udf = udf(calculate_time_after_id_creation, FloatType())
        df = df.withColumn("time_after_id_creation(hour)", calculate_time_after_id_creation_udf(df.last_access_time, df.first_access_time))
        df = df.withColumn("time_after_id_creation(hour)", round(col('time_after_id_creation(hour)')/1, 2))


        # convert timestamp to date (string)
        def create_numWeek(ts):
            return datetime.utcfromtimestamp(ts / 1000).strftime("%V")
        def create_numMonth(ts):
            return datetime.utcfromtimestamp(ts / 1000).strftime("%m")
        def create_numYear(ts):
            return datetime.utcfromtimestamp(ts / 1000).strftime("%Y")

        create_numWeek_udf = udf(create_numWeek, StringType())
        create_numMonth_udf = udf(create_numMonth, StringType())
        create_numYear_udf = udf(create_numYear, StringType())
        df = df.withColumn('num_week', create_numWeek_udf(col('ts')))
        df = df.withColumn('num_month', create_numMonth_udf(col('ts')))
        df = df.withColumn('num_year', create_numYear_udf(col('ts')))

        # Make a top_100 alltime artist list
        tmp_list = df.where(df.artist != "").groupby("artist").count().sort(col("count").desc()).collect()[0:100]
        top_100_alltime_artist_list = [row["artist"] for row in tmp_list]
        top_100_alltime_artist_list

        # Make udf to set 1 at churn column in every row of the chrun users 
        def create_top100_alltime(artist):
            if artist in top_100_alltime_artist_list:
                return 1
            else:
                return 0

        create_top100_alltime_udf = udf(create_top100_alltime, IntegerType())
        df = df.withColumn("top100_artist_alltime", create_top100_alltime_udf(df.artist))
        # Count total number of top 100
        w = Window.partitionBy(df.userId)
        df = df.withColumn('total_Top100_artist_alltime', sum('top100_artist_alltime').over(w))

        # Make a dictionary of a best seller song list of each week
        tmp_list = df.select("num_week").distinct().sort("num_week").collect()
        available_week_list = [row["num_week"] for row in tmp_list]
        available_week_list


    # Make a dictionary of a best seller song list of each week

        def create_dict_top100_song_week(available_week_list):
            dict_top100_song_week = dict()
            for week in available_week_list:
                top_100_week_song_list = df.where((df.artist != "") & (df.num_week == week)).groupby("song","num_week").count()\
                .sort(col("num_week"), col("count").desc()).collect()[0:100]
                top_100_week_song_list = [row['song'] for row in top_100_week_song_list]
                dict_top100_song_week[week] = top_100_week_song_list

            return dict_top100_song_week

        dict_top100_song_week = create_dict_top100_song_week(available_week_list)

        # Make a top_100_song_week list
        def create_top100_song_week(song, num_week):
            if song in dict_top100_song_week[num_week]:
                return 1
            else:
                return 0

        create_top100_song_week_udf = udf(create_top100_song_week, IntegerType())
        df = df.withColumn("top100_song_week", create_top100_song_week_udf(df.song, df.num_week))
        # Count total number of top 100_song_week
        w = Window.partitionBy(df.userId)
        df = df.withColumn('total_Top100_song_week', sum('top100_song_week').over(w))
        
        
        return df
    


    def subfunc_joinDf_featureExtraction(df):
        # Create other dataframes to be joined to df dataframe to get 
        # Number of thumb up/ thumb down/ advert/ add friend/ upgrade/ downgrade/ error/ logout
        thumbsup_df = df.where(df.page == 'Thumbs Up').groupBy("userId").agg(count("page").alias("total_thumbsup"))
        thumbsdown_df = df.where(df.page == 'Thumbs Down').groupBy("userId").agg(count("page").alias("total_thumbsdown"))
        advert_df =  df.where(df.page == 'Roll Advert').groupBy("userId").agg(count("page").alias("total_rolladvert"))
        addfriend_df =  df.where(df.page == 'Add Friend').groupBy("userId").agg(count("page").alias("total_addfriend"))
        addplaylist_df =  df.where(df.page == 'Add to Playlist').groupBy("userId").agg(count("page").alias("total_addplaylist"))
        sub_upgrade_df = df.where(df.page == 'Submit Upgrade').groupBy("userId").agg(count("page").alias("total_sub_upgrade"))
        sub_downgrade_df = df.where(df.page == 'Submit Downgrade').groupBy("userId").agg(count("page").alias("total_sub_downgrade"))
        error_df = df.where(df.page == 'Error').groupBy("userId").agg(count("page").alias("total_error"))
        logout_df = df.where(df.page == 'Logout').groupBy("userId").agg(count("page").alias("total_logout"))
        last_level_df = df[df.last_level != 'None'].select("userId","last_level")
        
        # Create total spent time
        spent_time_df = df.groupBy("userId", "sessionId").agg(max('ts').alias("max_ts_session"), 
                                                              min('ts').alias("min_ts_session")).orderBy("sessionId", ascending=True)
        def calculate_time_inSession(max_ts_session, min_ts_session):
            max_ts_session_datetime = datetime.utcfromtimestamp(max_ts_session / 1000)
            min_ts_session_datetime = datetime.utcfromtimestamp(min_ts_session / 1000)
            spent_time_session = max_ts_session_datetime - min_ts_session_datetime
            result = spent_time_session.total_seconds()/3600

            return result

        calculate_time_inSession_udf = udf(calculate_time_inSession, FloatType())
        spent_time_df = spent_time_df.withColumn("time_spent_Session_hour", 
                                                     calculate_time_inSession_udf(spent_time_df.max_ts_session, spent_time_df.min_ts_session))

        w = Window.partitionBy(spent_time_df.userId)
        spent_time_df = spent_time_df.withColumn("time_spent_Total_hour", sum(spent_time_df.time_spent_Session_hour).over(w))
        spent_time_df = spent_time_df.drop('max_ts_session','min_ts_session')
        spent_time_df = spent_time_df.withColumn('time_spent_Session_hour', round(col('time_spent_Session_hour')/1, 2))
        spent_time_df = spent_time_df.withColumn('time_spent_Total_hour', round(col('time_spent_Total_hour')/1, 2))
        spent_time_df_only_total = spent_time_df.drop('time_spent_Session_hour','sessionId').distinct()
        
        # Make df_new for machine learning
        df_new = df.select("userId",'gender', 'churn', 'os_system', 'location_first', 'total_sessionId', 'total_itemInSession',"time_after_id_creation(hour)", 
                           "total_Top100_artist_alltime", "total_Top100_song_week").dropna().drop_duplicates()
        df_new = df_new.join(thumbsup_df, 'userId', how='left').distinct()
        df_new = df_new.join(thumbsdown_df, 'userId', how='left').distinct()
        df_new = df_new.join(advert_df, 'userId', how='left').distinct()
        df_new = df_new.join(addfriend_df, 'userId', how='left').distinct()
        df_new = df_new.join(addplaylist_df, 'userId', how='left').distinct()
        df_new = df_new.join(sub_upgrade_df, 'userId', how='left').distinct()
        df_new = df_new.join(sub_downgrade_df, 'userId', how='left').distinct()
        df_new = df_new.join(error_df, 'userId', how='left').distinct()
        df_new = df_new.join(logout_df, 'userId', how='left').distinct()
        df_new = df_new.fillna(0, subset=['total_thumbsup','total_thumbsdown', 'total_rolladvert', 'total_addfriend', 'total_addplaylist', 'total_sub_upgrade', 
                                         'total_sub_downgrade', 'total_error', 'total_logout'])
        df_new = df_new.join(last_level_df, 'userId', how='left').distinct()
        df_new = df_new.join(spent_time_df_only_total, 'userId', how='left').distinct()
        
        df_new1 = df_new.withColumn("time_spent_Total_day", round(col('time_spent_Total_hour')/24, 2))
        df_new1 = df_new1.withColumn("time_after_id_creation(day)", round(col('time_after_id_creation(hour)')/24, 2))
        df_new1 = df_new1.withColumn("avg_total_sessionId_afterCreation", round(col('total_sessionId') / col('time_after_id_creation(day)'), 2))
        df_new1 = df_new1.withColumn("avg_itemInSession_afterCreation", round(col('total_itemInSession')/col('time_after_id_creation(day)'), 2))
        df_new1 = df_new1.withColumn("avg_thumbsup_afterCreation", round(col('total_thumbsup')/col('time_after_id_creation(day)'), 2))
        df_new1 = df_new1.withColumn("avg_thumbsdown_afterCreation", round((col('total_thumbsdown')/col('time_after_id_creation(day)')), 2))
        df_new1 = df_new1.withColumn("avg_rolladvert_afterCreation", round((col('total_rolladvert')/col('time_after_id_creation(day)')), 2))
        df_new1 = df_new1.withColumn("avg_addfriend_afterCreation", round((col('total_addfriend')/col('time_after_id_creation(day)')), 2))
        df_new1 = df_new1.withColumn("avg_addplaylist_afterCreation", round((col('total_addplaylist')/col('time_after_id_creation(day)')), 2))
        df_new1 = df_new1.withColumn("avg_error_afterCreation", round((col('total_error')/col('time_after_id_creation(day)')), 2))
        df_new1 = df_new1.withColumn("avg_logout_afterCreation", round((col('total_logout')/col('time_after_id_creation(day)')), 2))
        df_new1 = df_new1.withColumn("avg_total_Top100_artist_alltime", round((col('total_Top100_artist_alltime')/col('time_after_id_creation(day)')), 2))
        df_new1 = df_new1.withColumn("avg_total_Top100_song_week", round((col('total_Top100_song_week')/col('time_after_id_creation(day)')), 2))
        
        return df_new1
    
    df = subfunc_cleaning(df)
    print("Cleaning is done")
    df = subfunc_featureCreation(df)
    print("Feature creation is done")
    df_new1 = subfunc_joinDf_featureExtraction(df)
    print("Joining dataframes and feature extraction are done")
    
    return df_new1

df = spark.read.options(header=True).json("mini_sparkify_event_data.json")
df_ML = create_dataframe_forML(df)
df_ML.cache()
df_ML.take(1)

Cleaning is done
Feature creation is done
[Row(artist='Sleeping With Sirens', auth='Logged In', firstName='Darianna', gender='F', itemInSession=0, lastName='Carpenter', length=202.97098, level='free', location='Bridgeport-Stamford-Norwalk, CT', method='PUT', page='NextSong', registration=1538016340000, sessionId=31, song='Captain Tyin Knots VS Mr Walkway (No Way)', status=200, ts=1539003534000, userAgent='"Mozilla/5.0 (iPhone; CPU iPhone OS 7_1_2 like Mac OS X) AppleWebKit/537.51.2 (KHTML, like Gecko) Version/7.0 Mobile/11D257 Safari/9537.53"', userId='100010', churn=0, os_system='iPhone', location_first='CT', total_sessionId=7, total_itemInSession=381, last_access_time=1542823952000, first_access_time=1539003534000, last_level=None, time_after_id_creation(hour)=1061.23, num_week='41', num_month='10', num_year='2018', top100_artist_alltime=0, total_Top100_artist_alltime=61, top100_song_week=0, total_Top100_song_week=31)]
Joining dataframes and feature extraction are done


[Row(userId='100010', gender='F', churn=0, os_system='iPhone', location_first='CT', total_sessionId=7, total_itemInSession=381, time_after_id_creation(hour)=1061.23, total_Top100_artist_alltime=61, total_Top100_song_week=31, total_thumbsup=17, total_thumbsdown=5, total_rolladvert=52, total_addfriend=4, total_addplaylist=7, total_sub_upgrade=0, total_sub_downgrade=0, total_error=0, total_logout=5, last_level='free', time_spent_Total_hour=18.02, time_spent_Total_day=0.75, time_after_id_creation(day)=44.22, avg_total_sessionId_afterCreation=0.16, avg_itemInSession_afterCreation=8.62, avg_thumbsup_afterCreation=0.38, avg_thumbsdown_afterCreation=0.11, avg_rolladvert_afterCreation=1.18, avg_addfriend_afterCreation=0.09, avg_addplaylist_afterCreation=0.16, avg_error_afterCreation=0.0, avg_logout_afterCreation=0.11, avg_total_Top100_artist_alltime=1.38, avg_total_Top100_song_week=0.7)]

In [14]:
df_ML.count()

225

In [15]:
df_ML.head(1)

[Row(userId='100010', gender='F', churn=0, os_system='iPhone', location_first='CT', total_sessionId=7, total_itemInSession=381, time_after_id_creation(hour)=1061.23, total_Top100_artist_alltime=61, total_Top100_song_week=31, total_thumbsup=17, total_thumbsdown=5, total_rolladvert=52, total_addfriend=4, total_addplaylist=7, total_sub_upgrade=0, total_sub_downgrade=0, total_error=0, total_logout=5, last_level='free', time_spent_Total_hour=18.02, time_spent_Total_day=0.75, time_after_id_creation(day)=44.22, avg_total_sessionId_afterCreation=0.16, avg_itemInSession_afterCreation=8.62, avg_thumbsup_afterCreation=0.38, avg_thumbsdown_afterCreation=0.11, avg_rolladvert_afterCreation=1.18, avg_addfriend_afterCreation=0.09, avg_addplaylist_afterCreation=0.16, avg_error_afterCreation=0.0, avg_logout_afterCreation=0.11, avg_total_Top100_artist_alltime=1.38, avg_total_Top100_song_week=0.7)]